In [2]:
import numpy as np
import pandas as pd

import sklearn as sk

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold

In [21]:
ratings_table = pd.read_csv(filepath_or_buffer='../ml-1m/ratings.dat',
                            sep='::', delimiter=None, header=0, engine='python')

movies_table = pd.read_csv(filepath_or_buffer='../ml-1m/movies.dat',
                           sep='::', delimiter=None, header=0, engine='python')

users_table = pd.read_csv(filepath_or_buffer='../ml-1m/users.dat',
                          sep='::', delimiter=None, header=0, engine='python')


# 1.
mean_rating_global = ratings_table['Rating'].mean()
# 2.
mean_rating_per_movie = ratings_table.groupby('MovieID')['Rating'].mean()
# .3
mean_rating_per_user = ratings_table.groupby('UserID')['Rating'].mean()
print(mean_rating_per_user[1])


def create_n_dimensional_array(*features):
    X_set = np.vstack((features)).T

    return pd.DataFrame(X_set, columns=['UserID', 'MovieID'])

#TASK 1

# Add global 
# 1 average as a new column in the X data set
#2 the average rating per item
#3 the average rating per user
#4 and an “optimal”
#linear combination of the two averages (per user and per item), with and without
#the parameter γ.

#1 Global average

# X1 = create_n_dimensional_array(*[ratings_table['UserID'], ratings_table['MovieID']])


# for i in range(len(X1)):
#     np.append(X1[i], mean_rating_global)


#2 Average rating per item

X2 = create_n_dimensional_array(*[ratings_table['UserID'], ratings_table['MovieID']])

print(X2)
X2_features = X2.join(mean_rating_global, on=[])

print(X2_features)

for i in range(len(ratings_table['MovieID'])):
    X2[i].append(mean_rating_per_movie)

#3 the average rating per user

X3 = create_n_dimensional_array(*[ratings_table['UserID'], ratings_table['MovieID']])

for i in range(len(ratings_table['MovieID'])):
    X3[i].append(mean_rating_per_user)

#4 and an “optimal” #linear combination of the two averages (per user and per item), with and without the parameter γ.

X4 = create_n_dimensional_array(*[ratings_table['UserID'], ratings_table['MovieID']])

for i in range(len(ratings_table['MovieID'])):
    X4[i].append(mean_rating_per_movie, mean_rating_per_user)



y = ratings_table['Rating']

4.188679245283019
         UserID  MovieID
0             1     1193
1             1      661
2             1      914
3             1     3408
4             1     2355
...         ...      ...
1000204    6040     1091
1000205    6040     1094
1000206    6040      562
1000207    6040     1096
1000208    6040     1097

[1000209 rows x 2 columns]


ValueError: Joining multiple DataFrames only supported for joining on index

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

linear_reg = LinearRegression(fit_intercept=True)

LinearRegression( )

# train the model with the training data
linear_reg.fit(df_users_movies, y)



def k_fold_split_training(X, y): 
    
    X_total_train_data = []
    kf = KFold(n_splits=5, shuffle=True, random_state=32)
    # loop X_train[i] , i=[1, 5]
    for train_index, test_index in kf.split(X):
        
        
        
        # split the data into training & test -> (training data to train the model i.e create a linear line that fits the data)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        

        # initialize the linear regression model with gamma or other hyperparameters
        linear_reg = LinearRegression(fit_intercept=True)

        # train the model with the training data
        linear_reg.fit(X_train, y_train)
        
        
        y_predict = linear_reg.predict(X_test)
        
        
        
        # The coefficients
        print("Coefficients: \n", linear_reg.coef_)
        # The mean squared error
        print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predict))
        # The mean absolute error
        print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_predict))
        # The coefficient of determination: 1 is perfect prediction
        print("Coefficient of determination: %.2f" % r2_score(y_test, y_predict))
        

In [ ]:
# movies_users_merged = pd.merge(left=ratings_table['UserID'], right=ratings_table['MovieID'], on='UserID')

def pivot_table(data, values, index, column, fill_value=None):
    return pd.pivot_table(data=data,values=values, index=index, columns=column, fill_value=fill_value)


df_users_movies = pivot_table(data=ratings_table, values=['Rating'], index=['MovieID'], column=['UserID'], fill_value=mean_rating_global)

print(df_users_movies)
y = df_users_movies.to_numpy()